Importar librerías necesarias

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

Copia este archivo y coloca tus comentarios o notas en el código.

Configuración de dispositivo

Transformaciones de datos

Cargar el conjunto de datos CIFAR-10

In [2]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=4, shuffle=False)

100%|██████████| 170M/170M [00:12<00:00, 13.2MB/s]


Definición de la CNN

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=6, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128 * 3 * 3, 512)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.flatten(x)

        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

Inicialización del modelo, criterio y optimizador

In [8]:
net = Net().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Entrenamiento del modelo

In [9]:
train_losses = []
val_losses = []
val_accuracies = []
num_epochs = 5

print("--- Training Started ---")

for epoch in range(num_epochs):
    net.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(trainloader.dataset)
    train_losses.append(epoch_loss)

    net.eval()
    running_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            val_loss = loss_function(outputs, labels)
            running_val_loss += val_loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    epoch_val_loss = running_val_loss / len(testloader.dataset)
    val_losses.append(epoch_val_loss)

    epoch_accuracy = 100.0 * correct / total
    val_accuracies.append(epoch_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Accuracy: {epoch_accuracy:.2f}%")

print("--- Finished Training ---")

metrics = [train_losses, val_losses, val_accuracies]
print(metrics)

--- Training Started ---
Epoch [1/5], Train Loss: 1.6438, Val Loss: 1.2939, Val Accuracy: 54.61%
Epoch [2/5], Train Loss: 1.1909, Val Loss: 1.0344, Val Accuracy: 63.96%
Epoch [3/5], Train Loss: 1.0074, Val Loss: 0.9435, Val Accuracy: 67.25%
Epoch [4/5], Train Loss: 0.8883, Val Loss: 0.9181, Val Accuracy: 68.78%
Epoch [5/5], Train Loss: 0.8002, Val Loss: 0.8943, Val Accuracy: 69.67%
--- Finished Training ---
[[1.643791798388958, 1.1908757510781287, 1.0073684884829446, 0.8882555445211008, 0.8002155720828846], [1.293865408271551, 1.0344316519215704, 0.9434914861878381, 0.9181440514506772, 0.8942551290688104], [54.61, 63.96, 67.25, 68.78, 69.67]]
